## Загрузка данных и подключение библиотек

In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c tabular-playground-series-may-2021
!pip install catboost
!pip install eli5
!pip install optuna
!pip install shap
!pip install scikit-learn-extra
!unzip /content/test.csv.zip
!unzip /content/train.csv.zip
!unzip /content/sample_submission.csv.zip

kaggle.json
  0% 0.00/851k [00:00<?, ?B/s]
100% 851k/851k [00:00<00:00, 46.4MB/s]
  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 34.3MB/s]
  0% 0.00/1.72M [00:00<?, ?B/s]
100% 1.72M/1.72M [00:00<00:00, 103MB/s]
     |████████████████████████████████| 67.3MB 57kB/s 
     |████████████████████████████████| 112kB 12.7MB/s 
     |████████████████████████████████| 296kB 13.3MB/s 
     |████████████████████████████████| 1.2MB 23.6MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 112kB 31.7MB/s 
     |████████████████████████████████| 143kB 28.5MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for alembic: filename=alembic-1.6.2-py2.py3-none-any.whl size=164219 sha256=e69fb5213f3bff4035ba02b6588a9ea536782b7830f0d48522d5706152ca1241
  Stored in directory: /root/.cache/pip/wheels/a5/63/44/db29401e49ef5331c163b591f12a465c40af864bfa888dabd2
  Created wheel

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import catboost as cb
import sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA, FactorAnalysis as FA
from typing import List, Optional
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split
from typing import List, Tuple
import scipy.stats as ss
from sklearn_extra.cluster import KMedoids
import math
from sklearn.utils.validation import check_is_fitted
import eli5
from sklearn.base import BaseEstimator, TransformerMixin
import time
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import shap
import missingno as msno
from sklearn.inspection import permutation_importance
from eli5.sklearn import PermutationImportance
import optuna
from sklearn.metrics import log_loss
pd.plotting.register_matplotlib_converters()
%matplotlib inline
sns.set(color_codes=True)
pal = sns.color_palette("viridis", 10)
sns.set_palette(pal)

## Используемые функции

In [3]:
def get_input(data_path: str) -> pd.DataFrame:
  """
  Считывание данных и вывод основной информации о наборе данных.

  Parmeters
  ---------
  data_path: str - название файла

  Returns
  -------
  data: pandas.core.frame.DataFrame - загруженный набор данных в pandas.Dataframe
  """
  base_path = "/content"
  data = pd.read_csv(f"{base_path}/{data_path}")
  data.columns = [col.lower() for col in data.columns]
  print(f"{data_path}: shape = {data.shape[0]} rows, {data.shape[1]} cols")
  return data

In [4]:
def plot_feature_importance(importance, names, model_type, figsize=(10,8)):

  #Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=figsize)
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + ' FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')
  return list(fi_df['feature_names'].values)

In [5]:
def multi_estimators_predict(estimators: List,
                             x_valid: pd.DataFrame,
                             y_valid = None,
                             metric: callable = None,
                             scalers: List = None):
  preds = []
  evals = []
  if scalers:
    assert len(estimators) == len(scalers)

  for i in range(len(estimators)):
    if scalers:
      pred = estimators[i].predict_proba(scalers[i].transform(x_valid))
    else:
      if type(estimators[0]) == xgb.core.Booster:
        pred = estimators[i].predict(x_valid)
      else:
        pred = estimators[i].predict_proba(x_valid)

    
    preds.append(pred)
    if (y_valid is not None) and (metric is not None):
      eval = metric(y_valid, pred)
      evals.append(eval)
  
  result = np.stack([preds], axis=0)
  result = np.squeeze(result, axis=0)
  result = np.mean(result, axis=0)

  if (y_valid is not None) & (metric is not None):
    for i, eval in enumerate(evals):
      print(f"Model {i} metric: {eval:.7}")
    print(f"Result model metric: {metric(y_valid, result):.7}")

  return result

In [6]:
def catboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели catboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)], verbose=10, early_stopping_rounds=50,
            #cat_features=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [7]:
def lightgbm_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели lightgbm.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],
            eval_metric="multi_logloss", verbose=10, early_stopping_rounds=50,
            categorical_feature=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [70]:
def xgboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели xgboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    encoders: dict
        Список с объектами LabelEncoders.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, encoders = [], {}
    oof_preds = np.zeros((X.shape[0], 4))

    if categorical:
        for feature in categorical:
            encoder = LabelEncoder()
            X[feature] = encoder.fit_transform(X[feature].astype("str").fillna("NA"))
            encoders[feature] = encoder

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],
            eval_metric="mlogloss", 
            verbose=10, 
            early_stopping_rounds=50,
        )

        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        estimators.append(model)

    return estimators, encoders, oof_preds

## Загрузка данных и построение моделей

### lightgbm + optuna

In [ ]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])
    
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.3)
    dtrain = lgb.Dataset(train_x, label=train_y)
 
    #param = {
    #    'objective': 'multiclass',
    #    'metric': 'multi_logloss',
    #    'num_class': 4,
    #    'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
    #    'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
    #    'num_leaves': trial.suggest_int('num_leaves', 2, 256),
    #    'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
    #    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
    #    'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    #    'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    #}

    param = {
         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 30.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 30.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_uniform('subsample', 0,1),
        'learning_rate': trial.suggest_uniform('learning_rate', 0, 0.1 ),
        'max_depth': trial.suggest_int('max_depth', 1,100),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'min_child_weight' : trial.suggest_loguniform('min_child_weight' , 1e-5 , 1),
        'cat_smooth' : trial.suggest_int('cat_smooth', 1, 100),
        'cat_l2': trial.suggest_int('cat_l2',1,20),
        'metric': 'multi_logloss', 
        'random_state': 42,
        'num_class': 4,
        'objective': 'multiclass',
        
    }
 
    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(test_x)
    #pred_labels = np.rint(preds)
    log_loss = sklearn.metrics.log_loss(test_y, preds)
    return log_loss
 
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
 
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_trial)

[I 2021-05-16 18:25:20,684] A new study created in memory with name: no-name-882d6856-d646-41a5-a0ff-8f875f9c288c


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-16 18:25:37,726] Trial 0 finished with value: 1.0930728477741078 and parameters: {'reg_alpha': 0.0018382181760043421, 'reg_lambda': 0.0002564331226829421, 'colsample_bytree': 0.5, 'subsample': 0.11843285679715676, 'learning_rate': 0.034182391469566045, 'max_depth': 29, 'num_leaves': 229, 'min_child_samples': 162, 'min_child_weight': 0.00011634068652074863, 'cat_smooth': 44, 'cat_l2': 4}. Best is trial 0 with value: 1.0930728477741078.


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-16 18:25:51,617] Trial 1 finished with value: 1.0955685356214455 and parameters: {'reg_alpha': 1.6323695248442425, 'reg_lambda': 0.0036906669717957823, 'colsample_bytree': 0.8, 'subsample': 0.9637292807894524, 'learning_rate': 0.05302797282829857, 'max_depth': 30, 'num_leaves': 114, 'min_child_samples': 55, 'min_child_weight': 0.0002587880463853362, 'cat_smooth': 55, 'cat_l2': 15}. Best is trial 0 with value: 1.0930728477741078.


...

train.csv: shape = 100000 rows, 52 cols


[I 2021-05-16 18:52:17,499] Trial 98 finished with value: 1.097967890559086 and parameters: {'reg_alpha': 0.0010887442460382354, 'reg_lambda': 6.85770693776466e-05, 'colsample_bytree': 0.8, 'subsample': 0.02449513281945133, 'learning_rate': 0.042983706612790516, 'max_depth': 51, 'num_leaves': 182, 'min_child_samples': 214, 'min_child_weight': 0.00033639916015951063, 'cat_smooth': 8, 'cat_l2': 13}. Best is trial 50 with value: 1.088279019087586.


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-16 18:52:34,115] Trial 99 finished with value: 1.094605906603208 and parameters: {'reg_alpha': 0.0005789338442277418, 'reg_lambda': 2.447736393204762e-05, 'colsample_bytree': 0.8, 'subsample': 0.000486993365153488, 'learning_rate': 0.04565651204537804, 'max_depth': 39, 'num_leaves': 133, 'min_child_samples': 198, 'min_child_weight': 0.0009042688882242668, 'cat_smooth': 16, 'cat_l2': 4}. Best is trial 50 with value: 1.088279019087586.


Number of finished trials: 100
Best trial: {'reg_alpha': 0.9557349372478526, 'reg_lambda': 0.00565822311014884, 'colsample_bytree': 0.7, 'subsample': 0.7981326760346236, 'learning_rate': 0.04164055940276916, 'max_depth': 29, 'num_leaves': 622, 'min_child_samples': 150, 'min_child_weight': 0.0003566489059353313, 'cat_smooth': 38, 'cat_l2': 3}
Best value: FrozenTrial(number=50, values=[1.088279019087586], datetime_start=datetime.datetime(2021, 5, 16, 18, 37, 37, 365754), datetime_complete=datetime.datetime(2021, 5, 16, 18, 37, 56, 972100), params={'reg_alpha': 0.9557349372478526, 'reg_lambda': 0.00565822311014884, 'colsample_bytree': 0.7, 'subsample': 0.7981326760346236, 'learning_rate': 0.04164055940276916, 'max_depth': 29, 'num_leaves': 622, 'min_child_samples': 150, 'min_child_weight': 0.0003566489059353313, 'cat_smooth': 38, 'cat_l2': 3}, distributions={'reg_alpha': LogUniformDistribution(high=30.0, low=1e-05), 'reg_lambda': LogUniformDistribution(high=30.0, low=1e-05), 'colsample_by

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
print(study.best_params)

{'reg_alpha': 0.9557349372478526, 'reg_lambda': 0.00565822311014884, 'colsample_bytree': 0.7, 'subsample': 0.7981326760346236, 'learning_rate': 0.04164055940276916, 'max_depth': 29, 'num_leaves': 622, 'min_child_samples': 150, 'min_child_weight': 0.0003566489059353313, 'cat_smooth': 38, 'cat_l2': 3}


In [ ]:
study.best_value

1.088279019087586

In [11]:
data = get_input("train.csv")
test = get_input("test.csv")
sample_submission = get_input("sample_submission.csv")

data.drop(columns='id', inplace=True)

mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)

train, valid = train_test_split(
    data, train_size=0.7, shuffle=True, random_state=1,
)

train.reset_index(inplace=True, drop=True)
valid.reset_index(inplace=True, drop=True)

y_train = train['target']
x_train = train.drop(columns=['target'])
y_valid = valid['target']
x_valid = valid.drop(columns=['target'])

train.csv: shape = 100000 rows, 52 cols
test.csv: shape = 50000 rows, 51 cols
sample_submission.csv: shape = 50000 rows, 5 cols


In [34]:
lgbm_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 1000,
    "learning_rate": 0.12,
    "reg_lambda": 100,
    "n_jobs": -1,
    "seed": 27,
    'device_type': 'cpu',
    'num_class': 4,
    'random_state': 2,
    'lambda_l1': 1.044071701696505e-08, 
    'lambda_l2': 0.627862736041027, 
    'num_leaves': 16, 
    #"max_depth": 3,
    'feature_fraction': 0.9457112060526248, 
    'bagging_fraction': 0.6064976789336679, 
    'bagging_freq': 5, 
    'min_child_samples': 28
}

lgbm_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 1000,
    'reg_alpha': 0.9557349372478526, 
    'reg_lambda': 0.00565822311014884, 
    'colsample_bytree': 0.7, 
    'subsample': 0.7981326760346236, 
    'learning_rate': 0.04164055940276916, 
    'max_depth': 29, 
    'num_leaves': 622, 
    'min_child_samples': 150, 
    'min_child_weight': 0.0003566489059353313, 
    'cat_smooth': 38, 
    'cat_l2': 3
}

lgbm_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 5000,
    'reg_alpha': 11.159097442782404, 
    'reg_lambda': 6.133258909357973e-05, 
    'colsample_bytree': 0.3, 
    'subsample': 0.40110232869776463, 
    'learning_rate': 0.07591597509135133, 
    #'max_depth': 92, 
    #'num_leaves': 867,
    'max_depth': 8, 
    'num_leaves': 6, 
    'min_child_samples': 165, 
    'min_child_weight': 0.0029014788191160327, 
    'cat_smooth': 79, 
    'cat_l2': 3
}



cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

lgb_estimators, lgb_oof = lightgbm_cv_fit(
    lgbm_params, x_train, y_train, cv, #categorical=categorical_feature_names
)

Mon May 17 20:29:05 2021, Cross-Validation, 70000 rows, 50 cols
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's multi_logloss: 1.11448
[20]	valid_0's multi_logloss: 1.11139
[30]	valid_0's multi_logloss: 1.10881
[40]	valid_0's multi_logloss: 1.10694
[50]	valid_0's multi_logloss: 1.10521
[60]	valid_0's multi_logloss: 1.10365
[70]	valid_0's multi_logloss: 1.10248
[80]	valid_0's multi_logloss: 1.10133
[90]	valid_0's multi_logloss: 1.10036
[100]	valid_0's multi_logloss: 1.09952
[110]	valid_0's multi_logloss: 1.09876
[120]	valid_0's multi_logloss: 1.09804
[130]	valid_0's multi_logloss: 1.09746
[140]	valid_0's multi_logloss: 1.0969
[150]	valid_0's multi_logloss: 1.09631
[160]	valid_0's multi_logloss: 1.09581
[170]	valid_0's multi_logloss: 1.09532
[180]	valid_0's multi_logloss: 1.09502
[190]	valid_0's multi_logloss: 1.09464
[200]	valid_0's multi_logloss: 1.09434
[210]	valid_0's multi_logloss: 1.09407
[220]	valid_0's multi_logloss: 1.0938
[230]	valid_0's multi_loglo

In [35]:
print(f"Out of fold log loss {metrics.log_loss(y_train, lgb_oof)}")

Out of fold log loss 1.0927230279706703


In [36]:
result_lgbm = multi_estimators_predict(lgb_estimators, x_valid, y_valid, metrics.log_loss)

Model 0 metric: 1.090256
Model 1 metric: 1.089374
Model 2 metric: 1.090248
Model 3 metric: 1.090099
Model 4 metric: 1.090486
Model 5 metric: 1.089739
Model 6 metric: 1.090338
Result model metric: 1.089295


### Catboost

In [37]:
cb_params = {
    "n_estimators": 5000,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "CPU",
    'learning_rate': 0.08992656195498482, 
    'reg_lambda': 27.77866189042851, 
    'subsample': 0.6044483085727145, 
    'random_strength': 0.5855495663972144, 
    'depth': 3, 
    'min_data_in_leaf': 25, 
    'leaf_estimation_iterations': 1,
    'bootstrap_type': 'Bernoulli',
    'leaf_estimation_method': 'Newton',
}

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

cb_estimators, cb_oof = catboost_cv_fit(
    cb_params, x_train, y_train, cv, #categorical=categorical_feature_names
)

Mon May 17 20:32:36 2021, Cross-Validation, 70000 rows, 50 cols
0:	learn: 1.3351753	test: 1.3352115	best: 1.3352115 (0)	total: 86.7ms	remaining: 7m 13s
10:	learn: 1.1515302	test: 1.1518409	best: 1.1518409 (10)	total: 474ms	remaining: 3m 34s
20:	learn: 1.1204992	test: 1.1203623	best: 1.1203623 (20)	total: 908ms	remaining: 3m 35s
30:	learn: 1.1132077	test: 1.1130026	best: 1.1130026 (30)	total: 1.34s	remaining: 3m 35s
40:	learn: 1.1100112	test: 1.1100711	best: 1.1100711 (40)	total: 1.79s	remaining: 3m 36s
50:	learn: 1.1080076	test: 1.1080511	best: 1.1080511 (50)	total: 2.21s	remaining: 3m 34s
60:	learn: 1.1065421	test: 1.1066315	best: 1.1066315 (60)	total: 2.63s	remaining: 3m 32s
70:	learn: 1.1050763	test: 1.1053989	best: 1.1053989 (70)	total: 3.07s	remaining: 3m 33s
80:	learn: 1.1038282	test: 1.1043449	best: 1.1043449 (80)	total: 3.48s	remaining: 3m 31s
90:	learn: 1.1026597	test: 1.1033713	best: 1.1033713 (90)	total: 3.89s	remaining: 3m 29s
100:	learn: 1.1016839	test: 1.1026175	best: 1.1

In [38]:
print(f"Out of fold log loss {metrics.log_loss(y_train, cb_oof)}")

Out of fold log loss 1.092977742733346


In [39]:
result_cb = multi_estimators_predict(cb_estimators, x_valid, y_valid, metrics.log_loss)

Model 0 metric: 1.091101
Model 1 metric: 1.091115
Model 2 metric: 1.091027
Model 3 metric: 1.090776
Model 4 metric: 1.091148
Model 5 metric: 1.090821
Model 6 metric: 1.091873
Result model metric: 1.090472


### XGBoost

In [26]:
xgb_params = {
    'objective': 'multi:softprob',
    #'eval_metric': 'mlogloss',
    'random_state': 13,
    'max_depth': 5, 
    'n_estimators': 2000,
    'num_class': 4
}

xgb_params = {
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08356451010151393, 
    'gamma': 0.02911685058980812, 
    'max_depth': 5, 
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927, 
    'subsample': 0.6445037550866027, 
    'colsample_bytree': 0.07634753656242108, 
    'lambda': 13.663280761461781, 
    'alpha': 21.521205761694137, 
    'max_leaves': 48,
    'n_estimators': 2000,
    'num_class': 4
}

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

xgb_estimators, xgb_encoders, xgb_oof = xgboost_cv_fit(
    xgb_params, x_train, y_train, cv, #categorical=categorical_feature_names
)

Mon May 17 20:18:43 2021, Cross-Validation, 70000 rows, 50 cols
[0]	validation_0-mlogloss:1.35373
Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[10]	validation_0-mlogloss:1.18542
[20]	validation_0-mlogloss:1.13521
[30]	validation_0-mlogloss:1.11891
[40]	validation_0-mlogloss:1.11292
[50]	validation_0-mlogloss:1.11037
[60]	validation_0-mlogloss:1.1082
[70]	validation_0-mlogloss:1.10654
[80]	validation_0-mlogloss:1.10537
[90]	validation_0-mlogloss:1.10413
[100]	validation_0-mlogloss:1.10282
[110]	validation_0-mlogloss:1.10171
[120]	validation_0-mlogloss:1.10082
[130]	validation_0-mlogloss:1.10004
[140]	validation_0-mlogloss:1.09912
[150]	validation_0-mlogloss:1.09832
[160]	validation_0-mlogloss:1.09761
[170]	validation_0-mlogloss:1.09711
[180]	validation_0-mlogloss:1.09639
[190]	validation_0-mlogloss:1.09575
[200]	validation_0-mlogloss:1.09518
[210]	validation_0-mlogloss:1.09475
[220]	validation_0-mlogloss:1.09432
[230]	validation_0-mlogloss:1.09393
[240]	validatio

In [27]:
print(f"Out of fold log loss {metrics.log_loss(y_train, xgb_oof)}")

Out of fold log loss 1.0937984271524215


In [28]:
result_xgb = multi_estimators_predict(xgb_estimators, x_valid, y_valid, metrics.log_loss)

Model 0 metric: 1.092112
Model 1 metric: 1.091548
Model 2 metric: 1.0923
Model 3 metric: 1.092527
Model 4 metric: 1.09258
Model 5 metric: 1.092014
Model 6 metric: 1.091958
Result model metric: 1.090985


In [57]:
metrics.log_loss(y_valid, result_cb)

1.0904720673204586

In [58]:
metrics.log_loss(y_valid, result_xgb)

1.0909847912356256

In [59]:
metrics.log_loss(y_valid, result_lgbm)

1.089295294594471

In [62]:
result_lgb_xgb = np.dstack((result_lgbm, result_xgb))
result_lgb_xgb_mean = np.mean(result_lgb_xgb, axis=2)
print(f"result_lgb_xgb: {metrics.log_loss(y_valid, result_lgb_xgb_mean)}")

result_lgb_xgb: 1.0892976651266912


In [66]:
result_lgb_cb = np.dstack((result_lgbm, result_cb))
result_lgb_cb_mean = np.mean(result_lgb_cb, axis=2)
print(f"result_lgb_cb: {metrics.log_loss(y_valid, result_lgb_cb_mean)}")

result_lgb_cb: 1.089453366934431


In [67]:
result_xgb_cb = np.dstack((result_xgb, result_cb))
result_xgb_cb_mean = np.mean(result_xgb_cb, axis=2)
print(f"result_xgb_cb: {metrics.log_loss(y_valid, result_xgb_cb_mean)}")

result_xgb_cb: 1.0899651967669564


In [68]:
result_lgb_xgb_cb = np.dstack((result_lgbm, result_xgb, result_cb))
result_lgb_xgb_cb_mean = np.mean(result_lgb_xgb_cb, axis=2)
print(f"result_lgb_xgb_cb: {metrics.log_loss(y_valid, result_lgb_xgb_cb_mean)}")

result_lgb_xgb_cb: 1.0893458701040053


## Получение результата

In [69]:
data = get_input("train.csv")
test = get_input("test.csv")
data.drop(columns='id', inplace=True)
sample_submission = get_input("sample_submission.csv")

#data['sum'] = data[data.columns.to_list()[:50]].sum(axis=1) # норм
#data['max'] = data[data.columns.to_list()[:50]].max(axis=1) # норм
#data['not_nul_features'] = (data!=0)[data.columns.to_list()[:50]].sum(axis=1)
#data['2+13'] = data['feature_2'] + data['feature_13']
#data['6+15'] = data['feature_6'] + data['feature_15']

#test['sum'] = test[test.columns.to_list()[:50]].sum(axis=1) # норм
#test['max'] = test[test.columns.to_list()[:50]].max(axis=1) # норм
#test['not_nul_features'] = (test!=0)[test.columns.to_list()[:50]].sum(axis=1)
#test['2+13'] = test['feature_2'] + test['feature_13']
#test['6+15'] = test['feature_6'] + test['feature_15']

mapper = {'Class_1': 1, 'Class_2': 2, 'Class_3': 3, 'Class_4': 4}
data['target'] = data['target'].map(mapper)

y_train = data['target']
x_train = data.drop(columns=['target'])

train.csv: shape = 100000 rows, 52 cols
test.csv: shape = 50000 rows, 51 cols
sample_submission.csv: shape = 50000 rows, 5 cols


In [71]:
lgbm_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 5000,
    'reg_alpha': 11.159097442782404, 
    'reg_lambda': 6.133258909357973e-05, 
    'colsample_bytree': 0.3, 
    'subsample': 0.40110232869776463, 
    'learning_rate': 0.07591597509135133, 
    #'max_depth': 92, 
    #'num_leaves': 867,
    'max_depth': 8, 
    'num_leaves': 6, 
    'min_child_samples': 165, 
    'min_child_weight': 0.0029014788191160327, 
    'cat_smooth': 79, 
    'cat_l2': 3
}

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

lgb_estimators, lgb_oof = lightgbm_cv_fit(
    lgbm_params, x_train, y_train, cv, #categorical=categorical_feature_names
)

print(f"Out of fold log loss {metrics.log_loss(y_train, lgb_oof)}")
result_lgb = multi_estimators_predict(lgb_estimators, test.drop(columns='id'))

Mon May 17 20:51:57 2021, Cross-Validation, 100000 rows, 50 cols
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's multi_logloss: 1.11463
[20]	valid_0's multi_logloss: 1.11174
[30]	valid_0's multi_logloss: 1.10928
[40]	valid_0's multi_logloss: 1.10737
[50]	valid_0's multi_logloss: 1.10574
[60]	valid_0's multi_logloss: 1.10432
[70]	valid_0's multi_logloss: 1.10316
[80]	valid_0's multi_logloss: 1.10201
[90]	valid_0's multi_logloss: 1.10105
[100]	valid_0's multi_logloss: 1.10029
[110]	valid_0's multi_logloss: 1.09952
[120]	valid_0's multi_logloss: 1.0989
[130]	valid_0's multi_logloss: 1.09833
[140]	valid_0's multi_logloss: 1.09786
[150]	valid_0's multi_logloss: 1.09742
[160]	valid_0's multi_logloss: 1.09698
[170]	valid_0's multi_logloss: 1.09653
[180]	valid_0's multi_logloss: 1.09612
[190]	valid_0's multi_logloss: 1.09581
[200]	valid_0's multi_logloss: 1.09555
[210]	valid_0's multi_logloss: 1.09527
[220]	valid_0's multi_logloss: 1.09497
[230]	valid_0's multi_log

In [72]:
cb_params = {
    "n_estimators": 5000,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "CPU",
    'learning_rate': 0.08992656195498482, 
    'reg_lambda': 27.77866189042851, 
    'subsample': 0.6044483085727145, 
    'random_strength': 0.5855495663972144, 
    'depth': 3, 
    'min_data_in_leaf': 25, 
    'leaf_estimation_iterations': 1,
    'bootstrap_type': 'Bernoulli',
    'leaf_estimation_method': 'Newton',
}

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

cb_estimators, cb_oof = catboost_cv_fit(
    cb_params, x_train, y_train, cv, #categorical=categorical_feature_names
)

print(f"Out of fold log loss {metrics.log_loss(y_train, lgb_oof)}")
result_cb = multi_estimators_predict(cb_estimators, test.drop(columns='id'))

Mon May 17 20:57:58 2021, Cross-Validation, 100000 rows, 50 cols
0:	learn: 1.3351817	test: 1.3351765	best: 1.3351765 (0)	total: 58.3ms	remaining: 4m 51s
10:	learn: 1.1504047	test: 1.1507554	best: 1.1507554 (10)	total: 622ms	remaining: 4m 42s
20:	learn: 1.1191083	test: 1.1197820	best: 1.1197820 (20)	total: 1.22s	remaining: 4m 50s
30:	learn: 1.1116408	test: 1.1126188	best: 1.1126188 (30)	total: 1.86s	remaining: 4m 58s
40:	learn: 1.1087680	test: 1.1101258	best: 1.1101258 (40)	total: 2.48s	remaining: 4m 59s
50:	learn: 1.1066456	test: 1.1080972	best: 1.1080972 (50)	total: 3.09s	remaining: 4m 59s
60:	learn: 1.1050360	test: 1.1067681	best: 1.1067681 (60)	total: 3.67s	remaining: 4m 57s
70:	learn: 1.1036214	test: 1.1056447	best: 1.1056447 (70)	total: 4.26s	remaining: 4m 55s
80:	learn: 1.1022938	test: 1.1045768	best: 1.1045768 (80)	total: 4.86s	remaining: 4m 55s
90:	learn: 1.1011285	test: 1.1035656	best: 1.1035656 (90)	total: 5.45s	remaining: 4m 54s
100:	learn: 1.1001110	test: 1.1026694	best: 1.

In [73]:
xgb_params = {
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08356451010151393, 
    'gamma': 0.02911685058980812, 
    'max_depth': 5, 
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927, 
    'subsample': 0.6445037550866027, 
    'colsample_bytree': 0.07634753656242108, 
    'lambda': 13.663280761461781, 
    'alpha': 21.521205761694137, 
    'max_leaves': 48,
    'n_estimators': 2000,
    'num_class': 4
}

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

xgb_estimators, xgb_encoders, xgb_oof = xgboost_cv_fit(
    xgb_params, x_train, y_train, cv, #categorical=categorical_feature_names
)

print(f"Out of fold log loss {metrics.log_loss(y_train, lgb_oof)}")
result_xgb = multi_estimators_predict(xgb_estimators, test.drop(columns='id'))

Mon May 17 21:05:06 2021, Cross-Validation, 100000 rows, 50 cols
[0]	validation_0-mlogloss:1.35363
Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[10]	validation_0-mlogloss:1.18494
[20]	validation_0-mlogloss:1.13507
[30]	validation_0-mlogloss:1.11864
[40]	validation_0-mlogloss:1.11229
[50]	validation_0-mlogloss:1.10949
[60]	validation_0-mlogloss:1.10738
[70]	validation_0-mlogloss:1.10571
[80]	validation_0-mlogloss:1.10457
[90]	validation_0-mlogloss:1.10326
[100]	validation_0-mlogloss:1.10222
[110]	validation_0-mlogloss:1.10135
[120]	validation_0-mlogloss:1.10035
[130]	validation_0-mlogloss:1.09956
[140]	validation_0-mlogloss:1.09892
[150]	validation_0-mlogloss:1.09848
[160]	validation_0-mlogloss:1.09784
[170]	validation_0-mlogloss:1.09737
[180]	validation_0-mlogloss:1.09687
[190]	validation_0-mlogloss:1.09633
[200]	validation_0-mlogloss:1.09583
[210]	validation_0-mlogloss:1.09545
[220]	validation_0-mlogloss:1.0951
[230]	validation_0-mlogloss:1.09481
[240]	validati

In [74]:
result_lgb_xgb_cb = np.dstack((result_lgb, result_xgb, result_cb))
result_lgb_xgb_cb_mean = np.mean(result_lgb_xgb_cb, axis=2)

In [75]:
test_pred_df = pd.DataFrame(result_lgb_xgb_cb_mean)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)

In [76]:
result

,id,class_1,class_2,class_3,class_4
0,100000,0.090962,0.621724,0.165693,0.121621
1,100001,0.074952,0.701683,0.134995,0.088370
2,100002,0.083937,0.643118,0.175356,0.097588
3,100003,0.083970,0.535519,0.293697,0.086813
4,100004,0.074636,0.624492,0.185662,0.115210
...,...,...,...,...,...
49995,149995,0.091246,0.685393,0.163664,0.059698
49996,149996,0.077159,0.654197,0.131921,0.136723
49997,149997,0.081517,0.531436,0.226084,0.160963
49998,149998,0.081800,0.592620,0.166096,0.159484
